In [11]:


from __future__ import division

try:
    import matplotlib
    matplotlib.use('agg')
except ImportError:
    pass

import argparse
#import numpy as np
#import warnings 
import logging
import chainer.functions as F
#import certifi
#import chainer
from chainer import training
from chainer import cuda
from chainer.training import extensions
from chainer.training.triggers import ManualScheduleTrigger

#from chainercv.datasets import TransformDataset
#from chainercv.datasets import voc_detection_label_names
#from chainercv.datasets import VOCDetectionDataset
from chainercv.extensions import DetectionVOCEvaluator
#from chainercv.links import FasterRCNNVGG16
from chainercv.links.model.faster_rcnn import FasterRCNNTrainChain
#from chainercv import transforms
from chainer.datasets import TransformDataset

import chainercv

import numpy as np
import os
import warnings
import logging
#import xml.etree.ElementTree as ET
import scipy.io

import chainer

from chainercv.utils import read_image

In [2]:




class WIDERFACEDataset(chainer.dataset.DatasetMixin):

    def __init__(self, data_dir, label_mat_file,
                 use_difficult=False, return_difficult=False, 
                 exclude_file_list=None, logger=None):
        
        # id_list_file = os.path.join(
            # data_dir, 'ImageSets/Main/{0}.txt'.format(split))

        # self.ids = [id_.strip() for id_ in open(id_list_file)]

        self.data_dir = data_dir
        self.label_mat_file = label_mat_file
        self.use_difficult = use_difficult
        self.return_difficult = return_difficult
        
        self.logger = logger #for 
        # list up files
        mat = scipy.io.loadmat(self.label_mat_file)
        self.ids = []
        self.bboxs = {}
        self.labels = {}
        self.difficult = {}
        for i in range(len(mat['event_list'])):
            event = mat['event_list'][i,0][0]
            for j in range(len(mat['file_list'][i,0])):
                file = mat['file_list'][i,0][j,0][0]
                filename = "{}.jpg".format(file)
                filepath = os.path.join(data_dir, 'images', event, filename)
                if exclude_file_list != None and filename in exclude_file_list:
                    continue
                # bounding boxes and labels of the picture file
                bboxs = mat['face_bbx_list'][i,0][j,0]
                # convert from (x, y, w, h) to (y1, x1, y2, x2)
                swapped_bbxs = bboxs[:, [1,0,3,2]] #  (y,x,h,w)
                swapped_bbxs[:,2:4] = swapped_bbxs[:,2:4] + swapped_bbxs[:,0:2]
                
                invalid_labels = mat['invalid_label_list'][i,0][j,0].ravel()
                pose_labels = mat['pose_label_list'][i,0][j,0].ravel()
                illum_labels = mat['illumination_label_list'][i,0][j,0].ravel()
                occlusion_labels = mat['occlusion_label_list'][i,0][j,0].ravel()
                blur_labels = mat['blur_label_list'][i,0][j,0].ravel()
                expression_labels = mat['expression_label_list'][i,0][j,0].ravel()
                
                self.ids.append(filepath)
                self.bboxs[filepath] = swapped_bbxs.astype(np.float32)
                self.labels[filepath] = np.zeros(len(bboxs), dtype=np.int32) #dummy, always 0
                self.difficult[filepath] = invalid_labels

    def __len__(self):
        return len(self.ids)

    def get_example(self, i):
        """Returns the i-th example.
        Returns a color image and bounding boxes. The image is in CHW format.
        The returned image is RGB.
        Args:
            i (int): The index of the example.
        Returns:
            tuple of an image and bounding boxes
        """
        id_ = self.ids[i]
        bbox = self.bboxs[id_].astype(np.float32)
        label = self.labels[id_].astype(np.int32)
        difficult = self.difficult[id_].astype(np.bool)
        if not self.use_difficult:
            bbox = bbox[np.where(difficult==False)]
            label = label[np.where(difficult==False)]
            difficult = difficult[np.where(difficult==False)]

        # Load a image
        img_file = id_
        img = read_image(img_file, color=True)
        #print(img_file)
        if self.logger:
            self.logger.debug(img_file)
        if self.return_difficult:
            return img, bbox, label, difficult
        return img, bbox, label
        


In [26]:
def test():
    a = WIDERFACEDataset(WIDER_TRAIN_DIR, WIDER_TRAIN_ANNOTATION_MAT)
    id_ = r'/0--Parade/0_Parade_marchingband_1_100.jpg'
    bbx = a.bboxs[id_]
    lbl = a.labels[id_]
    import pdb; pdb.set_trace()

if __name__ == '__main__':
    test()

KeyError: '/0--Parade/0_Parade_marchingband_1_100.jpg'

In [23]:


#from wider_face_dataset import WIDERFACEDataset

# dataset paths
WIDER_TRAIN_DIR = '/Users/theco/faces/WIDER_train/WIDER_train/images'
WIDER_TRAIN_ANNOTATION_MAT = '/Users/theco/faces/wider_face_split/wider_face_split/wider_face_train.mat'
WIDER_VAL_DIR = '/Users/theco/faces/WIDER_val/WIDER_val/images'
WIDER_VAL_ANNOTATION_MAT = '/Users/theco/faces/wider_face_split/wider_face_split/wider_face_val.mat'

BLACKLIST_FILE = '/Users/theco/faces/blacklist.txt'

faster_rcnn = FasterRCNNVGG16(n_fg_class=1,
                              pretrained_model='imagenet')
                              #min_size=600, max_size=1000,)
def transform(in_data):
    img, bbox, label = in_data
    _, H, W = img.shape
    img = faster_rcnn.prepare(img)
    _, o_H, o_W = img.shape
    scale = o_H / H
    bbox = transforms.resize_bbox(bbox, (H, W), (o_H, o_W))

    # horizontally flip
    img, params = transforms.random_flip(
        img, x_random=True, return_param=True)
    bbox = transforms.flip_bbox(
        bbox, (o_H, o_W), x_flip=params['x_flip'])

    return img, bbox, label, scale

def main():
    #parser = argparse.ArgumentParser(
     #   description='ChainerCV training example: Faster R-CNN')
    #parser.add_argument('--gpu', '-g', type=int, default=-1)
    #parser.add_argument('--lr', '-l', type=float, default=1e-3)
    #parser.add_argument('--out', '-o', default='result',
    #                    help='Output directory')
    #parser.add_argument('--seed', '-s', type=int, default=0)
    #parser.add_argument('--step_size', '-ss', type=int, default=50000)
    #parser.add_argument('--iteration', '-i', type=int, default=70000)
    #parser.add_argument('--train_data_dir', '-t', default=WIDER_TRAIN_DIR,
    #                    help='Training dataset (WIDER_train)')
    #parser.add_argument('--train_annotation', '-ta', default=WIDER_TRAIN_ANNOTATION_MAT,
    #                    help='Annotation file (.mat) for training dataset')
    #parser.add_argument('--val_data_dir', '-v', default=WIDER_VAL_DIR,
    #                    help='Validation dataset (WIDER_train)')
    #parser.add_argument('--val_annotation', '-va', default=WIDER_VAL_ANNOTATION_MAT,
    #                    help='Annotation file (.mat) for validation dataset')
    #args = parser.parse_args()

    #np.random.seed(args.seed)
    np.random.seed(0)

    # for logging pocessed files
    logger = logging.getLogger('logger')
    logger.setLevel(logging.DEBUG)
    handler = logging.FileHandler(filename='filelog.log')
    handler.setLevel(logging.DEBUG)
    logger.addHandler(handler)
    print('logger')
    blacklist = []
    with open(BLACKLIST_FILE, 'r') as f:
        for line in f:
            l = line.strip()
            if l:
                blacklist.append(line.strip())
    
    # train_data = VOCDetectionDataset(split='trainval', year='2007')
    # test_data = VOCDetectionDataset(split='test', year='2007',
                                    # use_difficult=True, return_difficult=True)
    #train_data = WIDERFACEDataset(args.train_data_dir, args.train_annotation,
    #    logger=logger, exclude_file_list=blacklist)
    #test_data = WIDERFACEDataset(args.val_data_dir, args.val_annotation)

    train_data = WIDERFACEDataset(WIDER_TRAIN_DIR, WIDER_TRAIN_ANNOTATION_MAT,
        logger=logger, exclude_file_list=blacklist)
    test_data = WIDERFACEDataset(WIDER_VAL_DIR, WIDER_VAL_ANNOTATION_MAT)

    # faster_rcnn = FasterRCNNVGG16(n_fg_class=len(voc_detection_label_names),
                                  # pretrained_model='imagenet')
    faster_rcnn.use_preset('evaluate')
    model = FasterRCNNTrainChain(faster_rcnn)
    if 0 >= 0:
        model.to_gpu()
        #model.to_cpu()
        chainer.cuda.get_device().use()
        #chainer.cuda.get_device_from_array()
        #chainer.cuda.to_cpu()
    #optimizer = chainer.optimizers.MomentumSGD(lr=args.lr, momentum=0.9)
    optimizer = chainer.optimizers.MomentumSGD(lr=1e-3, momentum=0.9)
    optimizer.setup(model)
    optimizer.add_hook(chainer.optimizer.WeightDecay(rate=0.0005))


    train_data = TransformDataset(train_data, transform)
    #import pdb; pdb.set_trace()
    #train_iter = chainer.iterators.MultiprocessIterator(
    #    train_data, batch_size=1, n_processes=None, shared_mem=100000000)
    train_iter = chainer.iterators.SerialIterator(
        train_data, batch_size=1)
    test_iter = chainer.iterators.SerialIterator(
        test_data, batch_size=1, repeat=False, shuffle=False)
    updater = chainer.training.updater.StandardUpdater(
        train_iter, optimizer, device=0)

    trainer = training.Trainer(
        updater, (70000, 'iteration'), out='result')

    trainer.extend(
        extensions.snapshot_object(model.faster_rcnn, 'snapshot_model.npz'),
        trigger=(70000, 'iteration'))
    trainer.extend(extensions.ExponentialShift('lr', 0.1),
                   trigger=(50000, 'iteration'))

    log_interval = 20, 'iteration'
    plot_interval = 3000, 'iteration'
    print_interval = 20, 'iteration'

    trainer.extend(chainer.training.extensions.observe_lr(),
                   trigger=log_interval)
    trainer.extend(extensions.LogReport(trigger=log_interval))
    trainer.extend(extensions.PrintReport(
        ['iteration', 'epoch', 'elapsed_time', 'lr',
         'main/loss',
         'main/roi_loc_loss',
         'main/roi_cls_loss',
         'main/rpn_loc_loss',
         'main/rpn_cls_loss',
         'validation/main/map',
         ]), trigger=print_interval)
    trainer.extend(extensions.ProgressBar(update_interval=10))

    if extensions.PlotReport.available():
        trainer.extend(
            extensions.PlotReport(
                ['main/loss'],
                file_name='loss.png', trigger=plot_interval
            ),
            trigger=plot_interval
        )

    trainer.extend(
        DetectionVOCEvaluator(
            test_iter, model.faster_rcnn, use_07_metric=True,
            label_names=('face',)),
        trigger=ManualScheduleTrigger(
            [50000, 70000], 'iteration'))

    trainer.extend(extensions.dump_graph('main/loss'))

    #try:
        # warnings.filterwarnings('error', category=RuntimeWarning)
    trainer.run()
    #except RuntimeWarning as w:
    #    logger.debug(w)
    #    quit()

if __name__ == '__main__':
    main()


logger


Exception in main training loop: 'NoneType' object has no attribute 'ndim'
Traceback (most recent call last):
  File "C:\Users\theco\Anaconda3\lib\site-packages\chainer\training\trainer.py", line 343, in run
    update()
  File "C:\Users\theco\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 240, in update
    self.update_core()
  File "C:\Users\theco\Anaconda3\lib\site-packages\chainer\training\updaters\standard_updater.py", line 245, in update_core
    batch = iterator.next()
  File "C:\Users\theco\Anaconda3\lib\site-packages\chainer\iterators\serial_iterator.py", line 77, in __next__
    batch = [self.dataset[index] for index in indices]
  File "C:\Users\theco\Anaconda3\lib\site-packages\chainer\iterators\serial_iterator.py", line 77, in <listcomp>
    batch = [self.dataset[index] for index in indices]
  File "C:\Users\theco\Anaconda3\lib\site-packages\chainer\dataset\dataset_mixin.py", line 67, in __getitem__
    return self.get_example(index)
  File

AttributeError: 'NoneType' object has no attribute 'ndim'

In [3]:
import urllib.request
import os

def download_model(url, dest):
    destdir = os.path.dirname(dest)
    if not os.path.exists(destdir):
        os.makedirs(destdir)
    
    print("Downloading {}... \nThis may take several minutes.".format(dest))
    urllib.request.urlretrieve(url, dest)

In [13]:
import argparse
import matplotlib.pyplot as plot

import os
import chainer

from chainercv.links import FasterRCNNVGG16
from chainercv import utils
from chainercv.visualizations import vis_bbox

#from download_model import *
TRAINED_MODEL_DEFAULT = 'trained_model/snapshot_model.npz'
MODEL_URL = 'http://nixeneko.2-d.jp/hatenablog/20170724_facedetection_model/snapshot_model.npz'

# A pretrained model for the higher version of chainercv.
TRAINED_MODEL_NEW = 'trained_model/snapshot_model_20180404.npz'
MODEL_URL_NEW = 'http://nixeneko.2-d.jp/hatenablog/20170724_facedetection_model/snapshot_model_20180404.npz'

def main():
    chainer.config.train = False

    parser = argparse.ArgumentParser()
    parser.add_argument('--gpu', type=int, default=-1)
    parser.add_argument('--pretrained_model', default=TRAINED_MODEL_DEFAULT)
    parser.add_argument('image')
    args = parser.parse_args()

    if args.pretrained_model == TRAINED_MODEL_DEFAULT and \
       not os.path.exists(TRAINED_MODEL_DEFAULT):
        download_model.download_model(MODEL_URL, TRAINED_MODEL_DEFAULT)
    try:
        model = FasterRCNNVGG16(
            n_fg_class=1,
            pretrained_model=args.pretrained_model)
    
    # Workaround for the newer version of chainercv (maybe v0.7.0 and higher)
    except KeyError:
        if args.pretrained_model == TRAINED_MODEL_DEFAULT and \
           not os.path.exists(TRAINED_MODEL_NEW):
            download_model.download_model(MODEL_URL_NEW, TRAINED_MODEL_NEW)
            model = FasterRCNNVGG16(
                n_fg_class=1,
                pretrained_model=TRAINED_MODEL_NEW)
        else: 
            raise

    if args.gpu >= 0:
        model.to_gpu(args.gpu)
        chainer.cuda.get_device(args.gpu).use()

    img = utils.read_image(args.image, color=True)
    bboxes, labels, scores = model.predict([img])
    bbox, label, score = bboxes[0], labels[0], scores[0]

    vis_bbox(
        img, bbox, label, score, label_names=('face',))
    plot.show()


if __name__ == '__main__':
    main()


usage: ipykernel_launcher.py [-h] [--gpu GPU]
                             [--pretrained_model PRETRAINED_MODEL]
                             image
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\theco\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
%tb

SystemExit: 2

In [22]:
import cv2 
from PIL import Image
from numpy import asarray

In [15]:
im=Image.open(r'faces/WIDER_train/WIDER_train/images/3--Riot/3_Riot_Riot_3_1023.jpg')

In [ ]:
for image in faces:
    im=Image.open() #read next image

    new=asarray(im) #convert to numpy array
    
    
    
    
#print(new)

In [17]:
im.show()